# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121423e+02     1.492178e+00
 * time: 0.4185600280761719
     1     1.073075e+01     7.928360e-01
 * time: 1.0453720092773438
     2    -1.159505e+01     1.066299e+00
 * time: 1.1082050800323486
     3    -3.394802e+01     7.204887e-01
 * time: 1.2198121547698975
     4    -4.722604e+01     5.906748e-01
 * time: 1.2899680137634277
     5    -5.690557e+01     2.162229e-01
 * time: 1.3745582103729248
     6    -5.971635e+01     2.286221e-01
 * time: 1.4376230239868164
     7    -6.088355e+01     8.858131e-02
 * time: 1.5110390186309814
     8    -6.141556e+01     3.878360e-02
 * time: 1.563528060913086
     9    -6.170429e+01     2.856077e-02
 * time: 1.6157259941101074
    10    -6.188016e+01     3.496651e-02
 * time: 1.6682372093200684
    11    -6.199990e+01     2.126097e-02
 * time: 1.7209382057189941
    12    -6.208624e+01     1.938326e-02
 * time: 1.7880971431732178
    13    -6.215174e+01     1.345139e-02
 * time: 1.84034919738

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671080
    AtomicLocal         -18.8557942
    AtomicNonlocal      14.8522863
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485422 
    Xc                  -19.3336836

    total               -62.261666455861